In [1]:
import MetaTrader5 as mt5
import pandas as pd
import pytz
from datetime import datetime, timedelta
import time


In [2]:

# connect to MetaTrader 4 terminal
if not mt5.initialize(login=69380859, server="MetaQuotes-Demo",password="wwnspfp0"):
    print("initialize() failed, error code =",mt5.last_error())
    mt5.shutdown()


In [3]:

# set the instrument and timeframe
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

# set the start and end time for the data
end_time = datetime.now()- timedelta(days=0)
start_time = end_time - timedelta(days=65)

# convert the times to UTC
timezone = pytz.timezone("Etc/UTC")
start_time = timezone.localize(start_time)
end_time = timezone.localize(end_time)

# request the historical data from MetaTrader 4
rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
print(rates)

[(1681085340, 1.08953, 1.08961, 1.08953, 1.08961,  2, 130, 0)
 (1681085400, 1.08964, 1.08964, 1.08964, 1.08964,  1, 127, 0)
 (1681085460, 1.08955, 1.08955, 1.08955, 1.08955,  1, 136, 0) ...
 (1686586680, 1.07712, 1.07712, 1.07702, 1.07704, 38,   0, 0)
 (1686586740, 1.07704, 1.07705, 1.0769 , 1.07696, 48,   0, 0)
 (1686586800, 1.07696, 1.07703, 1.07689, 1.07692, 38,   0, 0)]


In [4]:

# convert the data to a Pandas DataFrame
mt4_new = pd.DataFrame(rates)
mt4_new['time'] = pd.to_datetime(mt4_new['time'], unit='s')
mt4_new.set_index('time', inplace=True)
mt4_new.drop(['spread', 'real_volume'], axis=1, inplace=True)

# print the first 5 rows of the data
print(mt4_new.head(), len(mt4_new))


                        open     high      low    close  tick_volume
time                                                                
2023-04-10 00:09:00  1.08953  1.08961  1.08953  1.08961            2
2023-04-10 00:10:00  1.08964  1.08964  1.08964  1.08964            1
2023-04-10 00:11:00  1.08955  1.08955  1.08955  1.08955            1
2023-04-10 00:12:00  1.08955  1.08955  1.08954  1.08954            2
2023-04-10 00:13:00  1.09005  1.09005  1.08954  1.08967            4 65405


In [8]:
try:
    mt4_current = pd.read_csv('backtest_data/mt4_eurusd_1m.csv', index_col=0, parse_dates=True)
    print("opening existintg dataframe")
except:
    print("creating empty dataframe")
    mt4_current = pd.DataFrame()

opening existintg dataframe


In [9]:
consolidated = pd.concat([mt4_current, mt4_new])
consolidated = consolidated[~consolidated.index.duplicated(keep='first')].sort_index()

In [10]:
consolidated.to_csv('mt4_eurusd.csv', index=True)

In [11]:

# disconnect from the MetaTrader 4 terminal
mt5.shutdown()


True